## Projeto de Previsão do Rendimento da Safra
Este notebook apresenta uma solução para o projeto, envolvendo:

- Análise Exploratória de Dados (EDA)
- Clusterização e Detecção de Outliers
- Construção de Modelos Preditivos de Regressão
- Avaliação dos Modelos

A base de dados utilizada é a “crop_yield.csv”, que contém informações sobre:

- Cultura
- Precipitação (mm dia 1)
- Umidade específica a 2 metros (g/kg)
- Umidade relativa a 2 metros (%)
- Temperatura a 2 metros (ºC)
- Rendimento (ton/ha)

### 1. Introdução e Carregamento dos Dados
Nesta seção, fazemos a leitura dos dados e apresentamos uma visão inicial do dataset.

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração de estilo para os gráficos
sns.set(style="whitegrid", palette="muted", color_codes=True)

# Carregando o dataset
df = pd.read_csv("crop_yield.csv")

# Exibindo as primeiras linhas do dataset
df.head()


Comentários:

- Verificamos as colunas disponíveis e uma amostra dos dados.
- Identificamos a necessidade de tratamento de valores faltantes ou outliers.

### 2. Análise Exploratória dos Dados (EDA)
Esta etapa inclui a análise descritiva e visualização das variáveis para entender a distribuição dos dados.

In [ ]:
# Informações gerais sobre o dataset
print(df.info())

# Estatísticas descritivas
print(df.describe())

# Distribuição das variáveis numéricas
df.hist(bins=20, figsize=(15, 10))
plt.suptitle("Histograma das Variáveis")
plt.show()

# Matriz de correlação para identificar relações entre as variáveis
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Matriz de Correlação")
plt.show()

# Boxplots para identificar outliers em cada variável numérica
plt.figure(figsize=(15, 6))
for idx, col in enumerate(['Precipitação (mm dia 1)', 'Umidade específica a 2 metros (g/kg)', 
                           'Umidade relativa a 2 metros (%)', 'Temperatura a 2 metros (ºC)', 'Rendimento']):
    plt.subplot(1, 5, idx+1)
    sns.boxplot(y=df[col])
    plt.title(col)
plt.tight_layout()
plt.show()


Comentários:

- A análise descritiva e os gráficos ajudam a identificar padrões, possíveis correlações e outliers que podem influenciar os modelos.

### 3. Clusterização e Detecção de Outliers
Utilizaremos métodos de clusterização para identificar grupos de rendimentos e detectar outliers. Neste exemplo, aplicaremos o K-Means e o DBSCAN.

#### Preparação dos Dados para Clusterização

from sklearn.preprocessing import StandardScaler

# Selecionando apenas as variáveis numéricas para clusterização (exceto 'Cultura')
features = df[['Precipitação (mm dia 1)', 'Umidade específica a 2 metros (g/kg)',
               'Umidade relativa a 2 metros (%)', 'Temperatura a 2 metros (ºC)', 'Rendimento']]

# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)


#### Aplicando o K-Means

In [ ]:
from sklearn.cluster import KMeans

# Definindo o número de clusters (podemos usar o método do cotovelo para identificar K ideal)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters_km = kmeans.fit_predict(X_scaled)

# Adicionando os clusters ao dataframe original
df['Cluster_KMeans'] = clusters_km

# Visualizando os clusters em um gráfico de dispersão (utilizando duas dimensões, por exemplo, Rendimento e Temperatura)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df['Temperatura a 2 metros (ºC)'], y=df['Rendimento'], hue=df['Cluster_KMeans'], palette="viridis")
plt.title("Clusterização com K-Means")
plt.show()


#### Aplicando o DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

# Parâmetros do DBSCAN (epsilon e min_samples) podem ser ajustados conforme a densidade dos dados
dbscan = DBSCAN(eps=0.8, min_samples=5)
clusters_db = dbscan.fit_predict(X_scaled)

# Adicionando os clusters do DBSCAN ao dataframe
df['Cluster_DBSCAN'] = clusters_db

# Visualizando os clusters DBSCAN
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df['Temperatura a 2 metros (ºC)'], y=df['Rendimento'], hue=df['Cluster_DBSCAN'], palette="deep")
plt.title("Clusterização com DBSCAN (Outliers marcados como -1)")
plt.show()


Comentários:

- O K-Means divide os dados em grupos pré-definidos, enquanto o DBSCAN também identifica pontos considerados ruído (marcados como –1).

### 4. Construção de Modelos Preditivos de Regressão
Serão desenvolvidos cinco modelos preditivos utilizando algoritmos distintos. A seguir, um exemplo com:

- Regressão Linear Simples / Múltipla
- Árvore de Decisão
- Random Forest
- XGBoost
- Regressão por Support Vector Regression (SVR)

#### Preparação dos Dados para Regressão

In [ ]:
from sklearn.model_selection import train_test_split

# Para predição, vamos utilizar todas as variáveis preditoras (exceto 'Cultura' e os indicadores de clusterização)
X = df[['Precipitação (mm dia 1)', 'Umidade específica a 2 metros (g/kg)',
        'Umidade relativa a 2 metros (%)', 'Temperatura a 2 metros (ºC)']]
y = df['Rendimento']

# Dividindo o dataset em treino (70%) e teste (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


#### Modelo 1: Regressão Linear

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Instanciando e treinando o modelo de regressão linear
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

# Previsões
y_pred_lr = model_lr.predict(X_test)

# Avaliação
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Regressão Linear:")
print(f"MAE: {mae_lr:.2f}, MSE: {mse_lr:.2f}, RMSE: {rmse_lr:.2f}, R²: {r2_lr:.2f}")


#### Modelo 2: Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeRegressor

model_dt = DecisionTreeRegressor(random_state=42)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)

mae_dt = mean_absolute_error(y_test, y_pred_dt)
mse_dt = mean_squared_error(y_test, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print("Árvore de Decisão:")
print(f"MAE: {mae_dt:.2f}, MSE: {mse_dt:.2f}, RMSE: {rmse_dt:.2f}, R²: {r2_dt:.2f}")


#### Modelo 3: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest:")
print(f"MAE: {mae_rf:.2f}, MSE: {mse_rf:.2f}, RMSE: {rmse_rf:.2f}, R²: {r2_rf:.2f}")


#### Modelo 4: XGBoost

In [ ]:
import xgboost as xgb

model_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost:")
print(f"MAE: {mae_xgb:.2f}, MSE: {mse_xgb:.2f}, RMSE: {rmse_xgb:.2f}, R²: {r2_xgb:.2f}")


#### Modelo 5: SVR (Support Vector Regression)

In [ ]:
from sklearn.svm import SVR

model_svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
model_svr.fit(X_train, y_train)
y_pred_svr = model_svr.predict(X_test)

mae_svr = mean_absolute_error(y_test, y_pred_svr)
mse_svr = mean_squared_error(y_test, y_pred_svr)
rmse_svr = np.sqrt(mse_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print("SVR:")
print(f"MAE: {mae_svr:.2f}, MSE: {mse_svr:.2f}, RMSE: {rmse_svr:.2f}, R²: {r2_svr:.2f}")


Comentários:

- Cada modelo é treinado e avaliado utilizando as métricas MAE, MSE, RMSE e R².
- A comparação entre os modelos permitirá identificar qual abordagem se ajusta melhor à previsão do rendimento.

### 5. Avaliação e Discussão dos Resultados
Nesta seção, os resultados dos modelos são comparados. Por exemplo:

In [ ]:
# Consolidando os resultados dos modelos em um DataFrame para comparação
resultados = pd.DataFrame({
    "Modelo": ["Regressão Linear", "Árvore de Decisão", "Random Forest", "XGBoost", "SVR"],
    "MAE": [mae_lr, mae_dt, mae_rf, mae_xgb, mae_svr],
    "MSE": [mse_lr, mse_dt, mse_rf, mse_xgb, mse_svr],
    "RMSE": [rmse_lr, rmse_dt, rmse_rf, rmse_xgb, rmse_svr],
    "R²": [r2_lr, r2_dt, r2_rf, r2_xgb, r2_svr]
})
resultados


Discussão:

- Analise os erros médios (MAE e RMSE) para identificar qual modelo apresenta previsões mais precisas.
- Considere o valor do R² para entender o quão bem cada modelo explica a variabilidade dos dados.
- Discuta possíveis razões para diferenças de desempenho e quais melhorias (como tuning de hiperparâmetros ou engenharia de features) podem ser implementadas.

### 6. Conclusões e Limitações
Conclusões:

- Integração de Técnicas: O projeto combinou abordagens não supervisionadas (clusterização) e supervisionadas (regressão) para entender e prever o rendimento agrícola.
- Resultados: Através dos modelos testados, foi possível identificar qual algoritmo teve melhor desempenho na previsão do rendimento, auxiliando na tomada de decisão para a gestão da produção.
- Outliers e Padrões: A identificação dos clusters e outliers contribuiu para a compreensão dos fatores que afetam a produtividade.

Limitações e Pontos de Melhoria:

- Possíveis limitações na qualidade dos dados (ruídos, valores faltantes) que podem influenciar os modelos.
- Necessidade de validação adicional com dados de outras safras ou de diferentes regiões para garantir a robustez dos modelos.
- Ajustes finos nos hiperparâmetros e testes com outras técnicas de engenharia de features podem melhorar o desempenho preditivo.